<a href="https://colab.research.google.com/github/h1chang/ufrgs-tcc-ncm-classifier/blob/main/code/TCC_Exploratory_Data_Analysis_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explorando os dados


[Portal Transparência - NF-e](https://portaldatransparencia.gov.br/download-de-dados/notas-fiscais).

Dúvidas / TODO:
- carga dos dados
  - métrica de similaridade entre bases (jan, fev, etc,...)

- balanceamento
  - remover repetições? ou repetições ajudam o treino?
  - remover classes com ocorrências < x
  - limitar classes em tamanho y
  
- estatística / visualização
  - dados faltantes (ncm sem ocorrência na amostra)
  - treemap





# Instalação de bibliotecas

In [ ]:
!pip install -U -q PyDrive
!pip install numpy
!pip install pandas
# !pip install vega_datasets
# !pip install altair
# !pip install geopandas

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
# import geopandas as gpd
# import chardet
import matplotlib.pyplot as plt
import os
import nltk
import seaborn as sns



In [ ]:
!pip install transformers datasets evaluate

# Carregando os dados

In [ ]:
#autenticação
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# carregando
directory_path = '/content/drive/MyDrive/Colab Notebooks/nf-e'

# todos arquivos da pasta
directory_files = os.listdir(directory_path)

# arquivo específico
# directory_files = ['202201_NFe_NotaFiscalItem.csv', '202202_NFe_NotaFiscalItem.csv', '202203_NFe_NotaFiscalItem.csv']
directory_files = ['202201_NFe_NotaFiscalItem.csv']

df = pd.DataFrame()
for file in directory_files:
    print('Reading file: ', file)
    df_file = pd.read_csv(os.path.join(directory_path, file), encoding = 'ISO-8859-1',low_memory=False)
    df = pd.concat([df, df_file])

# file = '202210_NFe_NotaFiscalItem_novo.csv'
# df_file = pd.read_csv(os.path.join(directory_path, file), encoding = 'ISO-8859-1',low_memory=False)
# df = pd.concat([df, df_file])

print(len(df))


# Explorando os dados

TODO:
1) repetição (uniques, valores únicos)
2) tamanho das descrições (word len)

## Domínio das colunas

In [ ]:
# visualizando nome das colunas
df.columns

In [ ]:
# Print the data types of all columns
print(df.dtypes)

Funções util para plotar informações

In [ ]:
def plot_hist(data, colum_name):
  # Create histogram
  plt.hist(data[colum_name], bins=100, edgecolor='black')

  # Set labels and title
  plt.xlabel('Value')
  plt.ylabel('Frequency')
  plt.title('Histogram')

  # Display the histogram
  plt.show()

def plot_bar(data):
  data.plot(kind='bar')

def box_plot(data):
  fig = plt.figure(figsize =(5, 5))

  # Creating plot
  plt.boxplot(data)

  # show plot
  plt.show()


def info_column(colum_name):
    print('Dados sobre a coluna: ', colum_name)
    unique_values = df[colum_name].nunique()
    print('Valores únicos: ', unique_values)

    value_counts = df[colum_name].value_counts()
    print('Top 20 valores em ordem decrescente: ')
    print(value_counts.nlargest(20))

    # plot_hist(value_counts.nlargest(20));
    plot_bar(value_counts.nlargest(20));
    # plot_bar(value_counts.nlowest(20));
    box_plot(value_counts)






In [ ]:
info_column('CÓDIGO NCM/SH')

In [ ]:
info_column('UNIDADE')

In [ ]:
info_column('DESCRIÇÃO DO PRODUTO/SERVIÇO')

# Processamento

## Mantém colunas de interesse

In [ ]:
# mantem colunas de interesse
main_columns = ['CÓDIGO NCM/SH', 'DESCRIÇÃO DO PRODUTO/SERVIÇO']

df = df[main_columns].copy()

## Códigos NCM inválidos

In [ ]:
# Count the number of values less than zero in the 'numbers' column
count_invalid_ncm = (df['CÓDIGO NCM/SH'] <= 0).sum()



print(len(df))
print(count_invalid_ncm)
print(count_invalid_ncm/len(df))

### Removendo NCM inválidas

In [ ]:
# Remove values less than zero from the 'numbers' column
df = df[df['CÓDIGO NCM/SH'] > 0]

## Descrições duplicadas

In [ ]:
df_unique = df.groupby(['CÓDIGO NCM/SH', 'DESCRIÇÃO DO PRODUTO/SERVIÇO']).ngroups
print(len(df))
print(df_unique)
print(df_unique/len(df))


### Removendo duplicatas

In [ ]:
df = df.drop_duplicates()
print(len(df))

## Word Count

In [ ]:
nltk.download('punkt')

#tokeniza o texto para verificar a distribuição de tokens por instância de treinamento
#nr_tokens tem o número de tokens da instância
df['nr_tokens'] = df['DESCRIÇÃO DO PRODUTO/SERVIÇO'].apply(lambda x: len(nltk.word_tokenize(x,language='portuguese')))

In [ ]:
print(f"Instâncias: {len(df)}")
print(f"Menor número de tokens: {df['nr_tokens'].min()}")
print(f"Maior número de tokens: {df['nr_tokens'].max()}")
print(f"Número médio de tokens: {df['nr_tokens'].mean():.1f}")
print(f"Instâncias com mais de 100 tokens: {len(df[df['nr_tokens']>100])}")
print(f"Instâncias com menos de 2 tokens: {len(df[df['nr_tokens']<2])}")
sns.displot(data=df, x="nr_tokens", kde=True)

In [ ]:
checkpoint = 'neuralmind/bert-base-portuguese-cased'
#importando o tokenizador
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

#fixando o tamanho da sequência (em número de tokens -- lembre que são subpalavras)
MAX_LENGHT = 512

#tokenizando o texto para verificar a distribuição de tokens por instância de treinamento
df['nr_tokens'] = df['DESCRIÇÃO DO PRODUTO/SERVIÇO'].apply(lambda x: len(tokenizer(x)['input_ids']))
print(f"Instâncias: {len(df)}")
print(f"Menor Número de tokens: {df['nr_tokens'].min()}")
print(f"Maior Número de tokens: {df['nr_tokens'].max()}")
print(f"Numero de tokens médio: {df['nr_tokens'].mean()}")
print(f"Instâncias com mais de {MAX_LENGHT} tokens: {len(df[df['nr_tokens']>MAX_LENGHT])}")
print(f"Instâncias com menos de 2 tokens: {len(df[df['nr_tokens']<2])}")
sns.displot(data=df, x="nr_tokens", kde=True)
plt.xlim(0, MAX_LENGHT)

## Criando coluna com ncm completa

In [ ]:
# criando coluna NCM string, adicionado padding (zeros a esquerda)
df['ncm_completa'] = df['CÓDIGO NCM/SH'].astype(str).str.zfill(8)

info_column('ncm_completa')


## Criando colunas para cada hierarquia

AA BB CC D E

01 23 45 6 7



*   AA = Capítulo
*   AA BB = Posição
*   AA BB CC = Subposição
*   AA BB CC D = Item
*   AA BB CC D E = Subitem

In [ ]:
df['capitulo'] = df['ncm_completa'].str.slice(0, 2)
df['capitulo'] = df['capitulo'].astype(int)

df['posicao'] = df['ncm_completa'].str.slice(0, 4)
df['posicao'] = df['posicao'].astype(int)

df['subposicao'] = df['ncm_completa'].str.slice(0, 6)
df['subposicao'] = df['subposicao'].astype(int)

df['item'] = df['ncm_completa'].str.slice(0, 7)
df['item'] = df['item'].astype(int)



In [ ]:
# info_column('capitulo')
# info_column('posicao')
# info_column('subposicao')
# info_column('item')

## Códigos Capítulo inválidos

In [ ]:
# count_invalid_capitulo = ((df['capitulo'] <= 0) & (df['capitulo'] > 97)).sum()
count_invalid_capitulo = ((df['capitulo'] <= 0) | (df['capitulo'] > 97)).sum()
print(len(df))
print(count_invalid_capitulo)
print(count_invalid_capitulo/len(df))

### Removendo Capítulos inválidos

In [ ]:

df = df[df['capitulo'] > 0]
df = df[df['capitulo'] <= 97]

## Salvando dados processados

In [ ]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/nf-e/output.csv', index=False)

## Salvando dados por capítulo

In [ ]:
df_capitulo = df[['DESCRIÇÃO DO PRODUTO/SERVIÇO', 'capitulo']]
df_capitulo.rename(columns={'DESCRIÇÃO DO PRODUTO/SERVIÇO': 'text'}, inplace=True)
df_capitulo.rename(columns={'capitulo': 'label'}, inplace=True)
print(df_capitulo)
df_capitulo.to_csv('/content/drive/MyDrive/Colab Notebooks/nf-e/df_capitulo.csv', index=False)